In [ ]:
#|default_exp token_classification

In [ ]:
#| export
import re
from functools import partial
from collections import defaultdict, Counter

from loguru import logger
from transformers import AutoTokenizer

In [ ]:
from ocrpostcorrection.icdar_data import InputToken

In [ ]:
#| export
bert_base_model_name = 'bert-base-multilingual-cased'

tokenizer = AutoTokenizer.from_pretrained(bert_base_model_name)

# Tokenization

In [ ]:
#| export
def tokenize_and_align_labels_with_tokenizer(tokenizer, examples):
    """Tokenize function, to be used as partial with instatiated tokenizer"""
    # Source: https://huggingface.co/docs/transformers/custom_datasets#token-classification-with-wnut-emerging-entities
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:                            # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:              # Only label the first token of a given word.
                label_ids.append(label[word_idx])

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

In [ ]:
#| export
def tokenize_and_align_labels(examples):
    """Function to tokenize samples and align the labels"""""
    return partial(tokenize_and_align_labels_with_tokenizer, tokenizer)

# Convert predictions into ICDAR output format

TODO: functions below need some fixing!

In [ ]:
#| export

def convert_predictions(samples, predictions, tokenizer):
    """Convert predictions for subwords into predictions for words"""
    #print('samples', len(samples))
    #print(samples)
    #print(samples[0].keys())
    #for sample in samples:
    #    print(sample.keys()) 

    tokenized_samples = tokenizer(samples["tokens"], truncation=True, is_split_into_words=True)
    #print(samples)

    #for sample in samples:
    #    print(sample.keys())
    
    # convert predictions to labels (label_ids)
    #p = np.argmax(pred.predictions, axis=2)
    #print(p)

    converted = defaultdict(dict)

    for i, (sample, preds) in enumerate(zip(samples, predictions)):
        #print(sample.keys())
        #label = sample['tags']
        #print(label)
        #print(len(preds), preds)
        word_ids = tokenized_samples.word_ids(batch_index=i)  # Map tokens to their respective word.
        #print(len(word_ids), word_ids)
        result = defaultdict(list)
        for word_idx, p_label in zip(word_ids, preds):
            #print(word_idx, p_label)
            if word_idx is not None:
                result[word_idx].append(p_label)
        
        new_tags = []
        for word_idx, preds in result.items():
            new_tag = 0
            c = Counter(preds)
            #print(c)
            if c[1] > 0 and c[1] >= c[2]:
                new_tag = 1
            elif c[2] > 0 and c[2] >= c[1]:
                new_tag = 2
            
            new_tags.append(new_tag)

        #print('pred', len(new_tags), new_tags)
        #print('tags', len(label), label)
        
        #print(sample)
        #print(sample['key'], sample['start_token_id'])
        converted[sample['key']][sample['start_token_id']] = new_tags

    return converted

In [ ]:
#| export

def extract_icdar_output(label_str, input_tokens):
    #print(label_str, input_tokens)
    #print(len(label_str), len(input_tokens))
    text_output = {}

    # Correct use of 2 (always following a 1)
    regex = r'12*'

    for match in re.finditer(regex, label_str):
        #print(match)
        #print(match.group())
        num_tokens = len(match.group())
        #print(match.start(), len(input_tokens))
        idx = input_tokens[match.start()].start
        text_output[f'{idx}:{num_tokens}'] = {}

    # Incorrect use of 2 (following a 0) -> interpret first 2 as 1
    regex = r'02+'

    for match in re.finditer(regex, label_str):
        #print(match)
        #print(match.group())
        num_tokens = len(match.group()) - 1
        idx = input_tokens[match.start()+1].start
        text_output[f'{idx}:{num_tokens}'] = {}
    
    return text_output

In [ ]:
#| hide
label_str = '1'
input_tokens = [InputToken(ocr='bal', gs='bla', start=0, len_ocr=3, label=1)]
output = extract_icdar_output(label_str, input_tokens)
assert output == {'0:1': {}}, output

In [ ]:
#| hide
label_str = '01'
input_tokens = [InputToken(ocr='one', gs='one', start=0, len_ocr=3, label=0),
                InputToken(ocr='tow', gs='two', start=4, len_ocr=3, label=1)]
output = extract_icdar_output(label_str, input_tokens)
assert output == {'4:1': {}}, output

In [ ]:
#| hide
label_str = '12'
input_tokens = [InputToken(ocr='one', gs='one', start=0, len_ocr=3, label=0),
                InputToken(ocr='tow', gs='two', start=4, len_ocr=3, label=1)]
output = extract_icdar_output(label_str, input_tokens)
assert output == {'0:2': {}}, output

In [ ]:
#| hide
label_str = '112'
input_tokens = [InputToken(ocr='one', gs='one', start=0, len_ocr=3, label=0),
                InputToken(ocr='one', gs='one', start=4, len_ocr=3, label=0),
                InputToken(ocr='tow', gs='two', start=8, len_ocr=3, label=1)]
output = extract_icdar_output(label_str, input_tokens)
assert output == {'0:1': {}, '4:2': {}}, output

In [ ]:
#| hide
label_str = '02'
input_tokens = [InputToken(ocr='one', gs='one', start=0, len_ocr=3, label=0),
                InputToken(ocr='tow', gs='two', start=4, len_ocr=3, label=1)]
output = extract_icdar_output(label_str, input_tokens)
assert output == {'4:1': {}}, output

In [ ]:
#| export

def generate_icdar_output(dataset, result, data_test):
    output = {}

    for key, preds in result.items():
        labels = defaultdict(list)
        #print(key)
        try:
            text = data_test[key]
            #print(len(text.input_tokens))
            #print(preds)
            for start, lbls in preds.items():
                for i, label in enumerate(lbls):
                    labels[int(start)+i].append(label)
            #print('LABELS')
            #print(labels)

            label_str = []

            for i, token in enumerate(text.input_tokens):
                #print(i, token, labels[i])
                if 2 in labels[i]:
                    label_str.append('2')
                elif 1 in labels[i]:
                    label_str.append('1')
                else:
                    label_str.append('0')
            label_str = ''.join(label_str)

            #print('LABEL STR')
            #print(label_str)

            output[key] = extract_icdar_output(label_str, text.input_tokens)
        except KeyError:
            logger.warning(f'No data found for text {key}')
    return output


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()